## How to convert a model from PyTorch to Tensorflow 
In this tutorial, we will describe how to go from PyTorch to Tensorflow using ONNX, an open ecosystem for interoperable AI models (https://github.com/onnx)  Thus, following the tutorials on https://github.com/onnx/tutorials we will convert our model from PyTorch to ONNX and then from ONNX to Tensorflow.

## **Installations**
First of all, we need to install ONNX and Tensorflow and the necessary packages.

**To install ONNX:**

> conda install -c conda-forge protobuf numpy \
pip install onnx

**To install Tensorflow:**

> pip install tensorflow-cpu \
(pip install tensorflow if you need also support for CUDA-enabled GPU cards)

Next **install onnx-tensorflow** by the following commands:

> git clone https://github.com/onnx/onnx-tensorflow.git  
cd onnx-tensorflow \
pip install -e .




## PyTorch Model
Starting from the model you have defined in PyTorch, you need to train and test it. After this, you should save the state of your net in a file, that will be used for the conversion.

In [ ]:
# PyTorch Model
pytorch_model = Net() 
# Train and test the model
...
# Save the trained model to a file
torch.save(pytorch_model.state_dict(), 'net_pytorch.pth')

## Export the trained model to ONNX
In order to export the model, Pytorch exporter needs to run the model once and save this resulting traced model to a ONNX file. Therefore, we need to provide an input (a random tensor with the right shape) for our model. 

In our case we consider a net that takes as inputs RGB images with shape (1, 3, 32, 32.)

ACHTUNG: If in your net there are average pooling layers (**AdaptiveAvgPool2d()**) (e.g. in the standard VGG), pay attention that this is not supported by ONNX. Thus you need to add the following flag in **torch.onnx.export()**
> operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK

In [ ]:
# Load the network
net_pytorch = Net()
net_pytorch.load_state_dict(torch.load('net_pytorch.pth'))

# Export the trained model to ONNX
dummy_input = torch.rand(torch.randn(1, 3, 32, 32)) # random input for the model
torch.onnx.export(net_pytorch, dummy_input, "net_onnx.onnx")

For a graph version of the onnx file you can use a ONNX viewer called Netron: https://github.com/lutzroeder/Netron.

## Import the ONNX model to Tensorflow


In [1]:
import onnx
from onnx_tf.backend import prepare

# Load the ONNX file
model_onnx = onnx.load('net_onnx.onnx')

# Import the ONNX model to Tensorflow
tf_rep = prepare(model_onnx)

ModuleNotFoundError: ignored

In order to understand if we are converting correctly the model, we can explore the *tf_rep* object return from *onnx.tf.backend.prepare*

In [ ]:
# Input nodes to the model
print('inputs:', tf_rep.inputs)

# Output nodes from the model
print('outputs:', tf_rep.outputs)

# All nodes in the model
print('tensor_dict:')
print(tf_rep.tensor_dict)

## Run the model in Tensorflow
After converting the model to Tensorflow, we can run it by taking an image with the right shape and format for our net.

In [ ]:
import numpy as np
from IPython.display import display
from PIL import Image

print('Image 1:')
img = Image.open('image.png').resize((32, 32))
array_img = np.array(img, dtype=np.float32)
print(array_img.shape)
array_img = array_img.reshape(1, 3, 32, 32)
print(array_img.shape)
output = tf_rep.run(array_img)


print('The image is classified as ', np.argmax(output))


## Save the Tensorflow model into a file

In [1]:
tf_rep.export_graph('net_tf.pb')

NameError: name 'tf_rep' is not defined

## Conversion into Tensorflow Lite
Conversion of the Tensorflow model to Tensorflow Lite. There exists some operations that are not supported in Tensorflow and also that are not present in Tensorflow Lite. You can find a complete lists of them at the following link:
https://www.tensorflow.org/lite/guide/ops_compatibility.
In order to overcome this difficulty (if you do not have restrictions on your running environment) you can uncomment some of the lines below to complet the conversion. You can read something more, also on how to include then these operations in your environment, here:
https://www.tensorflow.org/lite/guide/ops_select

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('net_tf.pb')
# If some unsupported operations by Tensorflow are present, uncomment those lines.
#converter.target_spec.supported_ops = [
#  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
#  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
#]

tflite_model = converter.convert()

# Save the model.
with open('net.tflite', 'wb') as f:
  f.write(tflite_model)
